In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import inspect
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [2]:
Base = declarative_base()

C:\Users\mh30f\AppData\Local\Temp\ipykernel_13492\4196137762.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Load data from "maindata.csv" into a DataFrame
maindata_df = pd.read_csv("Resource/maindata.csv")

In [4]:
maindata_df

,Country/Region,Total Confirmed Cases,Newly Confirmed Cases,Total Deaths,New Deaths,Total Recovered Cases,Newly Recovered Cases,ISO Code,Lat,Lon
0,Afghanistan,55696,16,2442,4,49281,195,AF,33.000000,65.000000
1,Albania,104313,986,1736,21,67158,849,AL,41.153300,20.168300
2,Algeria,112622,161,2973,3,77683,146,DZ,28.033900,1.659600
3,Andorra,10799,24,110,0,10356,37,AD,42.506300,1.521800
4,Angola,20695,55,502,1,19238,17,AO,-11.202692,17.873887
...,...,...,...,...,...,...,...,...,...,...
229,Wallis and Futuna,9,0,0,0,6,0,WF,-13.768752,-177.156097
230,Marshall Islands,4,0,0,0,4,0,MH,7.131474,171.184478
231,Vanuatu,1,0,0,0,1,0,VU,-15.376706,166.959158
232,Samoa,3,0,0,0,2,0,WS,-13.759029,-172.104629


In [5]:
# Create an SQLite database engine for the main database
engine = create_engine(f"sqlite:///output/countiesdata.sqlite")

In [6]:
# Define a class for the country data
class CountryData(Base):
    __tablename__ = 'country_data'
    id = Column(Integer, primary_key=True)
    country_name = Column(String(255))
    total_confirmed_cases = Column(Integer)
    newly_confirmed_cases = Column(Integer)
    total_deaths = Column(Integer)
    new_deaths = Column(Integer)
    total_recovered_cases = Column(Integer)
    newly_recovered_cases = Column(Integer)
    iso_code = Column(String(10))
    lat = Column(Float)
    lon = Column(Float)

In [7]:
# Create the table for country data in the main database
Base.metadata.create_all(engine)

# Create a session to work with the main database
session = Session(bind=engine)

In [8]:
# Iterate through the rows in the DataFrame
for index, row in maindata_df.iterrows():
    country = row['Country/Region']

    # Create a new instance of the CountryData class for the current country
    country_data_instance = CountryData(
        country_name=country,
        total_confirmed_cases=row['Total Confirmed Cases'],
        newly_confirmed_cases=row['Newly Confirmed Cases'],
        total_deaths=row['Total Deaths'],
        new_deaths=row['New Deaths'],
        total_recovered_cases=row['Total Recovered Cases'],
        newly_recovered_cases=row['Newly Recovered Cases'],
        iso_code=row['ISO Code'],
        lat=row['Lat'],
        lon=row['Lon']
    )

    # Add the instance to the main database session
    session.add(country_data_instance)

In [9]:
# Commit the changes to the main database
session.commit()

In [10]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()

['country_data']

In [11]:
# Close the session
session.close()